In [ ]:
# Setup

import os
from dotenv import load_dotenv

from gemini_batcher.gemini_batcher import GeminiBatcher
from gemini_batcher.input_handler.text_inputs import WebsiteInput
from gemini_batcher.input_handler.media_inputs import VideoFileInput

from gemini_batcher.gemini_config import GeminiConfig
from gemini_batcher.strategies import *

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [2]:
# Generating the content and questions to ask about.
# The content was gathered from a MIT lecture transcript which is accessible at https://ocw.mit.edu/courses/6-00-introduction-to-computer-science-and-programming-fall-2008/
# The questions were generated by Gemini using the transcript.

import httpx

questions = httpx.get("https://raw.githubusercontent.com/phil-daniel/gemini-batcher/refs/heads/main/examples/demo_files/questions.txt").text.split('\n')
content = WebsiteInput("https://raw.githubusercontent.com/phil-daniel/gemini-batcher/refs/heads/main/examples/demo_files/content.txt")

In [ ]:
# Setting the configuration of Gemini, these can be changed in the future.

config = GeminiConfig(
    api_key=GEMINI_API_KEY,
    model="gemini-2.0-flash",
)

# Creating the Gemini Batcher object.
client = GeminiBatcher(config)

# Setting the strategies you wish to use.
chunking_strategy = TextSlidingWindowChunking(
    chunk_char_size=30000,
    window_char_size=0
)

# Since the batch_size > len(batch) only 1 batch is used.
batching_strategy = FixedBatching(
    batch_size=50
)

In [ ]:
# Generates a list of answers to the inputted questions using the information in the content provided.
answers = client.generate_content(
    content=content,
    questions=questions,
    chunking_strategy=chunking_strategy,
    batching_strategy=batching_strategy,
)

print(f'Question 1: {questions[0]}')
print(f'Answer 1: {answers.content[questions[0]]}')
    

Question 1: What is the goal of MIT 6.00 (Introduction to Computer Science and Programming)?
Answer 1: To help everybody learn about computation.


Batching

In [5]:
# We can compare the number of tokens required when batching a different number of questions with one another.

chunking_strategy = TextSlidingWindowChunking(
    chunk_char_size=100000,
    window_char_size=0
)

for i in [1, 2, 5, 15, 30]:
    print(f'Batch size: {i}')
    batching_strategy = FixedBatching(batch_size=i)
    answers = client.generate_content(
        content=content,
        questions=questions,
        chunking_strategy=chunking_strategy,
        batching_strategy=batching_strategy,
        config=config
    )
    print(f'\tInput tokens: {answers.input_tokens}, Output tokens: {answers.output_tokens}')

Batch size: 1


	Input tokens: 401777, Output tokens: 956
Batch size: 2


	Input tokens: 201092, Output tokens: 1038
Batch size: 5
	Input tokens: 80681, Output tokens: 855
Batch size: 15
	Input tokens: 27167, Output tokens: 839
Batch size: 30
	Input tokens: 13787, Output tokens: 781
